# LC-NE Patch-seq Analysis

See also the interactive webapp at https://huggingface.co/spaces/hanhou/patchseq (wait for a minute when the app is loading).

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger()

pd.set_option('display.width', 1000)

# Ensure that the LCNE_patchseq_analysis package is installed
try:
    import LCNE_patchseq_analysis
except ImportError:
    !pip install LCNE_patchseq_analysis --quiet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the master table from Han's patch-seq pipeline

Including metadata, pinned CCF, extracted ephys metrics (both ipfx and eFEL), expression levels of selected genes, and mapmycells annotations.

In [114]:
from LCNE_patchseq_analysis.data_util.metadata import load_ephys_metadata
df_meta = load_ephys_metadata(if_from_s3=True, if_with_seq=True)
print(df_meta.shape)

# Show the latest updated data
print("The latest cell was updated on:", df_meta.sort_values("Date", ascending=False).Date.iloc[0])

INFO: Loading sequencing data from S3...
INFO: Loading sequencing data from https://aind-scratch-data.s3.us-west-2.amazonaws.com/aind-patchseq-data/seq/seq_preselected.csv
INFO: Successfully merged sequencing data for 333 out of 333 cells


(333, 370)
The latest cell was updated on: 2025-05-23


## Decide our filters

In [115]:
# Import the population analysis utilities
from LCNE_patchseq_analysis.population_analysis import create_filter_conditions, compute_confusion_matrix

# Create filter conditions using the refactored function
condition_mapper = create_filter_conditions(df_meta)
condition_mapper.keys()

dict_keys(['Fluorescence', 'Marker Gene Any Positive', 'Marker Gene All Positive', 'Marker Gene DBH Positive', 'Is DBH Subclass'])

In [116]:
# Compute confusion matrix using the refactored function
confusion_matrix = compute_confusion_matrix(condition_mapper, 'Fluorescence', 'Is DBH Subclass')
print(confusion_matrix)

`Fluorescence` does not have data: 4
`Is DBH Subclass` does not have data: 127
Any of them does not have data: 131
                    `Is DBH Subclass` (+)  `Is DBH Subclass` (-)
`Fluorescence` (+)                     47                     95
`Fluorescence` (-)                      5                     55


In [117]:
confusion_matrix = compute_confusion_matrix(condition_mapper, 'Fluorescence', 'Marker Gene DBH Positive')
print(confusion_matrix)

`Fluorescence` does not have data: 4
`Marker Gene DBH Positive` does not have data: 88
Any of them does not have data: 92
                    `Marker Gene DBH Positive` (+)  `Marker Gene DBH Positive` (-)
`Fluorescence` (+)                             128                              49
`Fluorescence` (-)                              32                              32


In [118]:
confusion_matrix = compute_confusion_matrix(condition_mapper, 'Marker Gene All Positive', 'Is DBH Subclass')
print(confusion_matrix)

`Marker Gene All Positive` does not have data: 88
`Is DBH Subclass` does not have data: 127
Any of them does not have data: 127
                                `Is DBH Subclass` (+)  `Is DBH Subclass` (-)
`Marker Gene All Positive` (+)                     51                     52
`Marker Gene All Positive` (-)                      1                    102
